In [1]:
pip install langchain chromadb huggingface_hub sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.8 MB/s e

In [3]:
pip install -U langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [8]:
pip install -U langchain langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.9 MB/s eta 0:00:00


In [12]:
pip install faiss-cpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 42.6 MB/s eta 0:00:00


In [5]:
# a sample file
corpus_text = """
Customer churn refers to the loss of customers or clients who stop doing business with a company or service.
It is a key metric for companies in subscription-based industries such as telecom, SaaS, and streaming services.
Common reasons for churn include high prices, poor customer service, lack of value, and better offers from competitors.
Churn can be voluntary (customer chooses to leave) or involuntary (due to failed payments, etc).
Customer retention strategies include loyalty programs, personalized offers, proactive support, and satisfaction surveys.
In the telecom industry, predicting churn allows companies to take preventive actions before the customer leaves.
Machine learning models like Logistic Regression, Random Forest, and XGBoost are commonly used to build churn prediction systems.
These models use features like tenure, monthly charges, contract type, internet service, and customer support calls.
One popular dataset for churn modeling is the Telco Customer Churn dataset, which includes features like gender, contract type, and payment method.
Churn rate is calculated as: (Number of customers lost in a period / Total customers at the start) * 100.
Reducing churn improves lifetime value (LTV), profitability, and customer satisfaction.
It is cheaper to retain an existing customer than to acquire a new one.
Predictive churn models must be updated regularly as customer behavior patterns change over time.
Explainable AI (XAI) tools like SHAP or LIME can help interpret the predictions of black-box churn models.
Integrating churn prediction into CRM systems enables targeted marketing campaigns and better decision-making.
"""

with open("corpus.txt", "w") as f:
    f.write(corpus_text)


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

# Load corpus
loader = TextLoader("corpus.txt")
documents = loader.load()

# Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Use free HuggingFace embeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embedding)

# Save it
vectorstore.save_local("faiss_store")


In [22]:
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Load the FAISS vector store from disk
vectorstore = FAISS.load_local("faiss_store", embeddings=embedding, allow_dangerous_deserialization=True)

# Convert it to a retriever
retriever = vectorstore.as_retriever()

# Load a lightweight Hugging Face LLM
hf_pipeline = pipeline(
    model="MBZUAI/LaMini-Flan-T5-783M",
    task="text2text-generation"
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Set up conversational memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create the Conversational Retrieval QA Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [23]:
query = "What does customer churn mean?"
response = qa_chain.run(query)
print(response)


Customer churn refers to the loss of customers or clients who stop doing business with a company or service.


In [24]:
query = "What are common reasons for customer churn?"
response = qa_chain.run(query)
print(response)


The common reasons for customer churn are high prices, poor customer service, lack of value, and better offers from competitors.
